<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/alex-works/Colabs/GridNumba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
#@title Baixando pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost

In [2]:
! git init
! git remote add origin https://github.com/alessandronascimento/pyLiBELa.git
! git pull origin alex-works

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/alessandronascimento/pyLiBELa
 * branch            alex-works -> FETCH_HEAD
Already up to date.


In [3]:
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make -j4 

mv: cannot stat 'pyLiBELa/src': No such file or directory
--2023-05-23 17:25:32--  https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5606 (5.5K) [text/plain]
Saving to: ‘Makefile’

Makefile            100%[===================>]   5.47K  --.-KB/s    in 0s      

2023-05-23 17:25:32 (65.6 MB/s) - ‘Makefile’ saved [5606/5606]

g++ -fPIC -O3 -I/usr/local/include/python3.10 -I/usr/include/openbabel3 -I/usr/include/eigen3 -DBUILD=0 -DHAVE_EIGEN -fopenmp -Wunused-value src/pyPARSER.cpp -c -o obj/pyPARSER.o
g++ -fPIC -O3 -I/usr/local/include/python3.10 -I/usr/include/openbabel3 -I/usr/include/eigen3 -DBUILD=0 -DHAVE_EIGEN -fopenmp -Wunused-value src/pyMol2.cpp -c -o obj/pyMol2.o
g++ -

In [ ]:
!rm -rf obj

# Initializations

In [29]:
#@title Importando bibliotecas {display-mode: "form"}

from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyConformer import *
from pyRAND import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *
from pyMC import *
from pyFullSearch import *
from pyDocker import *

from tqdm import tqdm
import numpy as np
import numba as nb
from numba import jit
import numba.cuda as cuda
import pdb
import math
import cmath

In [5]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 10 #@param {type:"number"}
Input.search_box_y = 10 #@param {type:"number"}
Input.search_box_z = 10 #@param {type:"number"}
Input.grid_spacing = 1.0 #@param {type:"number"}
Input.x_dim = 5 #@param {type:"number"}
Input.y_dim = 5 #@param {type:"number"}
Input.z_dim = 5 #@param {type:"number"}
Input.write_grids = True;
Input.parallel_jobs = 2

# Para que funcione a funcão

Input.dielectric_model = "constant" #@param {type: "string"}

In [9]:
Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "test/lig.mol2.gz")
rec = Mol2(Input, "test/rec.mol2.gz")
Energy = Energy2(Input)
com = Coord.compute_com(lig)

In [51]:
refgrid = Grid(Input, Writer, rec, com)
py_grid = Grid(Input, Writer, rec, com)
numba_grid = Grid(Input, Writer, rec, com)

In [ ]:
for i in range(len(rec.charges)):
  
  print(rec.charges[i], '\n')

In [ ]:
refgrid.compute_grid_softcore(rec)

In [ ]:
refgrid.compute_grid_softcore_HB_omp(rec)

In [39]:
del(refgrid)

In [40]:
del(py_grid)

In [41]:
del(numba_grid)

In [ ]:
for i in range(grid.npointsy):
  print('\n')
  for j in range(grid.npointsz):
    print(f'{refgrid.rec_solv_gauss[0][i][j]:.2f}', end=' ')

# Porting work

In [50]:
#@title compute_softcore_HB_omp


def compute_grid_softcore_HB_omp(rec, grid, input):
  
  #if input.scoring_function >= 2:
  #  compute_grid_hardcore_HB_omp(...)
  #  return

  HB_C12 = np.double(55332.873)
  HB_C10 = np.double(18393.199)

  for i in range(grid.npointsx):
    x = np.double(i*grid.grid_spacing + grid.xbegin) 

    for j in range(grid.npointsy):
      y = np.double(j*grid.grid_spacing + grid.ybegin) 

      for k in range(grid.npointsz):
        z = np.double(k*grid.grid_spacing + grid.zbegin) 

        elec = np.double(0.0)
        vdwA = np.double(0.0)
        vdwB = np.double(0.0) 
        solv = np.double(0.0) 
        rec_solv = np.double(0.0) 
        hb_donor = np.double(0.0) 
        hb_acceptor = np.double(0.0) 

        for a in range(rec.N):
          
          d2 = np.double(grid.distance_squared(x, rec.xyz[a][0], y, rec.xyz[a][1], z, rec.xyz[a][2]))
          d6 = np.double(np.power(d2, 3))
          
          denom = np.double(0.0)
            
          if input.dielectric_model == "constant":
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a])/(input.diel*denom)
            solv += ((input.solvation_alpha * rec.charges[a] * rec.charges[a]) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))

          else:
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a]/(input.diel*denom))
            solv += ((input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
          
          denom = (d6 + input.deltaij6)
          vdwA += (4096*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 6)) / np.power(denom, 2)
          vdwB += ( 128*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 3)) / denom

        for m in range(len(rec.HBdonors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBdonors[m][1]][0], x, rec.xyz[rec.HBdonors[m][1]][1], y, rec.xyz[rec.HBdonors[m][1]][2], z)
          d10 = np.double(np.power(d2, 5))
          ang = np.double(grid.angle(rec.xyz[rec.HBdonors[m][0]][0], rec.xyz[rec.HBdonors[m][0]][1], rec.xyz[rec.HBdonors[m][0]][2], 
                          rec.xyz[rec.HBdonors[m][1]][0], rec.xyz[rec.HBdonors[m][1]][1], rec.xyz[rec.HBdonors[m][1]][2], x, y, z))
          angle_term = np.double(np.power(np.cos(ang * np.pi / 180), 4))
          hb_donor += ((HB_C12/(d10*d2)) - (HB_C10/d10)) * angle_term

        for n in range(len(rec.HBacceptors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBacceptors[n]][0], x, rec.xyz[rec.HBacceptors[n]][1], y, rec.xyz[rec.HBacceptors[n]][2], z)
          d10 = np.double(np.power(d2, 5))
          hb_acceptor += ((HB_C12/(d10*d2)) - (HB_C10/d10))

        grid.elec_grid[i][j][k] = elec
        grid.vdwA_grid[i][j][k] = vdwA
        grid.vdwB_grid[i][j][k] = vdwB
        grid.solv_gauss[i][j][k] = solv
        grid.rec_solv_gauss[i][j][k] = rec_solv
        grid.hb_donor_grid[i][j][k] = hb_donor
        grid.hb_acceptor_grid[i][j][k] = hb_acceptor;

  grid.rec_si = 0.0
  for a in range(rec.N):
    grid.rec_si += (input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta


In [28]:
@jit(nopython = True)
def distance_squared(x1, x2, y1, y2, z1, z2):
  return math.pow(x2 - x1, 2) + math.pow(y2 - y1, 2) + math.pow(z2 - z1, 2)

In [34]:
@jit(nopython = True)
def angle(x1, y1, z1, x2, y2, z2, x3, y3, z3):
  ab = math.sqrt(distance_squared(x1, x2, y1, y2, z1, z2))
  ac = math.sqrt(distance_squared(x1, x3, y1, y3, z1, z3))
  bc = math.sqrt(distance_squared(x2, x3, y2, y3, z2, z3))

  angle = math.acos((math.pow(ab, 2) + math.pow(bc, 2) - math.pow(ac, 2))/(2*ab*bc))
  return angle * 180 / np.pi

In [57]:
#@title (NUMBA) _compute_softcore_HB_omp_numba

#from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
#import warnings

#warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
#warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)


@jit(nopython=True)
def _compute_grid_softcore_HB_omp_numba(npointsx, npointsy, npointsz,  # from Grid here onwards
                                   grid_spacing, 
                                   xbegin, ybegin, zbegin,
                                   dielectric_model, # from Parser here onwards 
                                   deltaij_es6, deltaij6,
                                   solvation_alpha, solvation_beta,
                                   sigma, diel,
                                   N, # from Mol2 here onwards
                                   xyz = np.empty((0,0), dtype=np.float64), 
                                   charges = np.empty(0, dtype=np.float64), 
                                   radii = np.empty(0, dtype=np.float64), 
                                   epsilons_sqrt = np.empty(0, dtype=np.float64), 
                                   HBdonors = np.empty((0,0), dtype=np.int64), 
                                   HBacceptors = np.empty(0, dtype=np.int64), 
                                   out_elec_grid = np.empty((0,0,0), dtype=np.float64), # output arrays from here onwards 
                                   out_vdwA_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_vdwB_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_solv_gauss = np.empty((0,0,0), dtype=np.float64),
                                   out_rec_solv_gauss = np.empty((0,0,0), dtype=np.float64),
                                   out_hb_donor_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_hb_acceptor_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_rec_si = np.empty(1), dtype=np.float64):
  
  #if input.scoring_function >= 2:
  #  compute_grid_hardcore_HB_omp(...)
  #  return

  HB_C12 = 55332.873
  HB_C10 = 18393.199

  for i in range(npointsx):
    x = i*grid_spacing + xbegin

    for j in range(npointsy):
      y = j*grid_spacing + ybegin

      for k in range(npointsz):
        z = k*grid_spacing + zbegin 

        elec = 0.0
        vdwA = 0.0
        vdwB = 0.0 
        solv = 0.0 
        rec_solv = 0.0 
        hb_donor = 0.0 
        hb_acceptor = 0.0 

        for a in range(N):
          
          d2 = distance_squared(x, xyz[a][0], y, xyz[a][1], z, xyz[a][2])
          d6 = math.pow(d2, 3)
          
          denom = 0.0
            
          if dielectric_model == "constant":
            denom = math.pow(d6 + deltaij_es6, 1/3)
            elec += (332.0*charges[a])/(diel*denom)
            solv += ((solvation_alpha * charges[a] * charges[a]) + solvation_beta) * math.exp((-denom/(2 * math.pow(sigma, 2)))) / (math.pow(sigma, 3))
            rec_solv += (4/3) * np.pi * math.pow(radii[a], 3) * math.exp((-denom/(2*math.pow(sigma, 2)))) / (math.pow(sigma, 3))

          else:
            denom = math.pow(d6 + deltaij_es6, 1/3)
            elec += (332.0*charges[a]/(diel*denom))
            solv += ((solvation_alpha * math.pow(charges[a], 2)) + solvation_beta) * math.exp((-denom/(2 * math.pow(sigma, 2)))) / (math.pow(sigma, 3))
            rec_solv += (4/3) * np.pi * math.pow(radii[a], 3) * math.exp((-denom/(2*math.pow(sigma, 2)))) / (math.pow(sigma, 3))
          
          denom = (d6 + deltaij6)
          vdwA += (4096*epsilons_sqrt[a] * math.pow(radii[a], 6)) / math.pow(denom, 2)
          vdwB += ( 128*epsilons_sqrt[a] * math.pow(radii[a], 3)) / denom

        for m in range(len(HBdonors)):
          HBd0 = HBdonors[m][0]
          HBd1 = HBdonors[m][1]
          d2 = distance_squared(xyz[HBd1][0], x, xyz[HBd1][0], y, xyz[HBd1][0], z)
          d10 = math.pow(d2, 5)
          ang = angle(xyz[HBdonors[m][0]][0], xyz[HBdonors[m][0]][1], xyz[HBdonors[m][0]][2],
                          xyz[HBdonors[m][1]][0], xyz[HBdonors[m][1]][1], xyz[HBdonors[m][1]][2], x, y, z)
          angle_term = np.double(math.pow(math.cos(ang * np.pi / 180), 4))
          hb_donor += ((HB_C12/(d10*d2)) - (HB_C10/d10)) * angle_term

        for n in range(len(HBacceptors)):
          d2 = distance_squared(xyz[HBacceptors[n]][0], x, xyz[HBacceptors[n]][1], y, xyz[HBacceptors[n]][2], z)
          d10 = math.pow(d2, 5)
          hb_acceptor += ((HB_C12/(d10*d2)) - (HB_C10/d10))

        out_elec_grid[i][j][k] = elec
        out_vdwA_grid[i][j][k] = vdwA
        out_vdwB_grid[i][j][k] = vdwB
        out_solv_gauss[i][j][k] = solv
        out_rec_solv_gauss[i][j][k] = rec_solv
        out_hb_donor_grid[i][j][k] = hb_donor
        out_hb_acceptor_grid[i][j][k] = hb_acceptor;

  out_rec_si[0] = 0.0
  for a in range(N):
    out_rec_si[0] += (solvation_alpha * math.pow(charges[a], 2)) + solvation_beta


In [49]:
#@title (Wrapper) compute_softcore_HB_omp_numba
def compute_grid_softcore_HB_omp_numba(rec, grid, input):
  """Wrapper for calling the njit function"""

  out_elec_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_vdwA_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_vdwB_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_solv_gauss = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_rec_solv_gauss = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_hb_donor_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_hb_acceptor_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_rec_si = np.empty((1))

  HBdonors = np.array(rec.HBdonors)
  if not np.size(HBdonors):
    HBdonors = np.empty((0,0), dtype=np.int64)

  _compute_grid_softcore_HB_omp_numba(grid.npointsx, grid.npointsy, grid.npointsz, 
                                  grid.grid_spacing,
                                  grid.xbegin, grid.ybegin, grid.zbegin,
                                  input.dielectric_model,
                                  input.deltaij_es6, input.deltaij6,
                                  input.solvation_alpha, input.solvation_beta,
                                  input.sigma, input.diel,
                                  rec.N,
                                  np.array(rec.xyz),
                                  np.array(rec.charges),
                                  np.array(rec.radii),
                                  np.array(rec.epsilons_sqrt),
                                  HBdonors,
                                  np.array(rec.HBacceptors, dtype=np.int64),
                                  out_elec_grid,
                                  out_vdwA_grid,
                                  out_vdwB_grid,
                                  out_solv_gauss,
                                  out_rec_solv_gauss,
                                  out_hb_donor_grid,
                                  out_hb_acceptor_grid,
                                  out_rec_si)
  
  grid.rec_si = out_rec_si[0]

  # TODO: think about the best way of doing this copy operation
  for i in range(grid.npointsx):
    for j in range(grid.npointsy):
      for k in range(grid.npointsz):
        grid.elec_grid[i][j][k] = out_elec_grid[i][j][k]
        grid.vdwA_grid[i][j][k] = out_vdwA_grid[i][j][k]
        grid.vdwB_grid[i][j][k] = out_vdwB_grid[i][j][k]
        grid.solv_gauss[i][j][k] = out_solv_gauss[i][j][k]
        grid.rec_solv_gauss[i][j][k] = out_rec_solv_gauss[i][j][k]
        grid.hb_donor_grid[i][j][k] = out_hb_donor_grid[i][j][k]
        grid.hb_acceptor_grid[i][j][k] = out_hb_acceptor_grid[i][j][k]
  



In [ ]:
arr = np.array(rec.xyz)
print(arr)

In [27]:
compute_grid_softcore_HB_omp(rec, py_grid, Input)

In [34]:
compute_grid_softcore_HB_omp_numba(rec, numba_grid, Input)

#Print grids

In [ ]:
for i in range(py_grid.npointsx):
  print('\n------------------------')
  for j in range(py_grid.npointsy):
    print('\n')
    for k in range(py_grid.npointsz):
      print(f'{numba_grid.elec_grid[i][j][k]:.2f}', end=' ')

In [ ]:
refgrid.compute_grid_hardcore_HB_omp(rec)

In [ ]:
for i in range(refgrid.npointsx):
  print('\n------------------------')
  for j in range(refgrid.npointsy):
    print('\n')
    for k in range(refgrid.npointsz):
      print(f'{refgrid.elec_grid[i][j][k]:.2f}', end=' ')

#Benchmarking

In [52]:
print(f'search_box_x : {Input.search_box_x} \t search_box_y : {Input.search_box_y} \t search_box_z : {Input.search_box_z}')
print(f'x_dim : {Input.x_dim} \t y_dim : {Input.y_dim} \t z_dim : {Input.z_dim}')
print(f'grid_spacing : {Input.grid_spacing}\n')

#%timeit refgrid.compute_grid_hardcore_HB_omp(rec) # versão C++
#%timeit compute_grid_softcore_HB_omp(rec, py_grid, Input) # versão python usando objetos expostos pelo boost
%timeit compute_grid_softcore_HB_omp_numba(rec, numba_grid, Input) # versão numba usando njit

search_box_x : 10.0 	 search_box_y : 10.0 	 search_box_z : 10.0
x_dim : 5.0 	 y_dim : 5.0 	 z_dim : 5.0
grid_spacing : 1.0

65 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Misc

In [ ]:
@jit
def foo(npointsx):
  for i in range(npointsx):
    n = i**2

In [ ]:
foo(grid.npointsx)
foo.inspect_types()

In [ ]:
test_arr = np.size(np.array([1]))
bool(test_arr)

True

In [ ]:
#testing

print(refgrid.angle(0, 0, 0, 4, 5, 6, 7, 8, 9))
print(angle(0, 0, 0, 4 ,5 ,6 ,7 ,8 ,9))
#angle.inspect_types()

In [ ]:
#testing
print(refgrid.distance_squared(1, 2, 3, 4, 5, 6))
print(distance_squared(1, 2, 3, 4, 5, 6))

In [35]:
print(refgrid.rec_si)
print(numba_grid.rec_si)

29.797074292591553
29.797074292591553


# Whiteboard

Preparativos para rodar `_compute_grid_softcore_HB_omp()` usando numba.cuda \:

Será um kernel único?

sobre os inputs: 
  >organizar os escalares em arrays estruturados por tipo?
tipos : int64, float64 e char* (str : dieletric_model) -> cuda tem limitações com strings -> ler à respeito ou simplemente criar uma função intermediária e transformá-lo em enum.
  
  >talvez seja algo a considerar utilizar int32 e float32 para ser melhor compatível com a arquitetura da GPU, não sei se podem ocorrer truncagens. Verificar.

  >tenho que criar unit tests para comparar a precisão dos resultados sendo produzidos pelas funções que estou portando com as do C++ qualquer maneira. Se a precisão não for o maior detrimento, talvez podemos utilizar a flag nomath=True. Verificar.

  >saída: escrever em arrays como já feito? Tem um possível overhead a se considerar com as cópias sendo feitas (não só entre host-device, device-host, mas depois a host-host por meio da cópia elemento-a-elemento para os arrays do objetos).

sobre lógica interna do kernel:
  O loop mais interno `for a in range(N)` vai ter que ser rodado por cada thread mesmo e será efetivamente o ponto inicial do kernel.

sobre a hierarquia de threads:
  >organizar threads em 3d é o caminho mais óbvio para este problema. Como os grids dos ligantes provavelmente serão cubos em sua maioria, creio que faça sentido organizar os threadblocks em (10,10,10) (?) (supondo que o número de threads deve ser 1024, mas melhor verificar se tem uma forma mais programática de determinar isso no ambiente do colab).


outras considerações:

>como as duas funções chamadas dentro de `_compute_grid_softcore_HB_omp()` são simples, provavelmente dá para torná-las _device functions_ https://numba.readthedocs.io/en/stable/cuda/device-functions.html 

>me parece que não será necessário atomics, a função não parece ter casos de racing conditions.

>talvez streams sejam uma boa ideia, mas acho melhor avaliar depois de criar uma versão preliminar, checando a performance _as-is_. 

>ainda não sei se é aplicável a utilização de variáveis em shared memory. Que nem streams, avaliarei depois de ter uma implementação preliminar.




